In [38]:
import os
!pip install datasets transformers librosa soundfile numpy"<"2 "torch==2.2.1" torchaudio


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Obtaining dependency information for torch==2.2.1 from https://files.pythonhosted.org/packages/9e/4b/19527706ab1d1f91cdafe0160407a61e5865e74592ea911f92a81c583125/torch-2.2.1-cp312-none-macosx_10_9_x86_64.whl.metadata
INFO: pip is looking at multiple versions of torchaudio to determine which version is compatible with other requirements. This could take a while.
  Obtaining dependency information for torchaudio from https://files.pythonhosted.org/packages/b5/5b/105c9d2ff5257262d97c3b35d4740d242b673557d577959e9aae025a9cab/torchaudio-2.2.1-cp312-cp312-macosx_10_13_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.8/150.8 MB 3.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 7.0 MB/s eta 0:00:0000:0100:01m
  Attempting uninstall: torch
    Found existing installation: torch 2.2.2
    Uninstalling torch-2.2.2:
      Successfully uninstalled torch-2.2.2
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 2.2.2


In [1]:
%load_ext autoreload
%autoreload 2

from datasets import load_dataset
from transformers import pipeline

dataset = load_dataset("ashraq/esc50")
audio = dataset["train"]["audio"][-1]["array"]


Repo card metadata block was not found. Setting CardData to empty.


# old

In [23]:
from transformers import ClapModel
clap_model = ClapModel.from_pretrained("laion/clap-htsat-fused")

In [8]:
audio_classifier = pipeline(task="zero-shot-audio-classification", model="laion/clap-htsat-fused", device='cpu')
output = audio_classifier(audio, candidate_labels=["Sound of a dog", "Sound of vaccum cleaner"])
print(output)

Device set to use cpu


[{'score': 0.9998761415481567, 'label': 'Sound of a dog'}, {'score': 0.00012389849871397018, 'label': 'Sound of vaccum cleaner'}]


In [6]:
#audio.shape
audio_classifier(audio[0:2205], candidate_labels=["Sound of a dog", "Sound of vaccum cleaner"])


[{'score': 0.7010166645050049, 'label': 'Sound of vaccum cleaner'},
 {'score': 0.2989833652973175, 'label': 'Sound of a dog'}]

In [9]:
audio_classifier.feature_extractor.sampling_rate

48000

In [17]:
import transformers
import numpy as np
isinstance(audio, np.ndarray)
#type(audio_classifier)
features: transformers.feature_extraction_utils.BatchFeature = audio_classifier.feature_extractor(
            [audio], 
    sampling_rate=audio_classifier.feature_extractor.sampling_rate, 
    return_tensors="pt"
)

transformers.feature_extraction_utils.BatchFeature

In [21]:
features.data['input_features'].shape


torch.Size([1, 4, 1001, 64])

In [22]:
type(audio_classifier.feature_extractor), type(audio_classifier.model)

(transformers.models.clap.feature_extraction_clap.ClapFeatureExtractor,
 transformers.models.clap.modeling_clap.ClapModel)

In [9]:
from transformers import ZeroShotAudioClassificationPipeline
from transformers import ClapModel

audio_classifier: ZeroShotAudioClassificationPipeline
model = audio_classifier.model: ClapAudioModel


ClapModel(
  (text_model): ClapTextModel(
    (embeddings): ClapTextEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ClapTextEncoder(
      (layer): ModuleList(
        (0-11): 12 x ClapTextLayer(
          (attention): ClapTextAttention(
            (self): ClapTextSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ClapTextSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm):

# Setup

In [1]:
%load_ext autoreload
%autoreload 2

device='mps'
from transformers import ClapModel, ClapFeatureExtractor
model = ClapModel.from_pretrained("laion/clap-htsat-unfused", use_safetensors=True).to(device)
feature_extractor: ClapFeatureExtractor = ClapFeatureExtractor.from_pretrained("laion/clap-htsat-unfused")


# Run

In [2]:
import torchaudio
waveform, sampling_rate = torchaudio.load("/Users/damian/Downloads/green-day-when-i-come-around-4k-upgrade.mp3")
print(sampling_rate, waveform.shape)
#inputs = feature_extractor(waveform, sampling_rate=sample_rate, return_tensors="pt")
#audio_features = model.get_audio_features(**inputs)

48000 torch.Size([2, 8631136])


In [3]:
inputs = feature_extractor(waveform[0, 0:48000], sampling_rate=sample_rate, return_tensors="pt")
audio_features = model.get_audio_features(**inputs)
audio_features

tensor([[ 3.8695e-03, -1.0465e-02,  5.2422e-03, -6.0487e-02,  4.6928e-03,
          5.2191e-02, -1.4813e-02, -2.1933e-03,  6.5858e-02, -3.7286e-03,
         -2.1416e-02, -3.3354e-02,  1.0374e-02,  1.1024e-02,  3.1634e-02,
          2.3485e-02,  2.6002e-02, -5.8393e-02,  9.3072e-03,  4.0768e-02,
          1.3185e-02,  3.3300e-02, -4.9509e-02, -1.5833e-01,  6.1356e-03,
          3.3087e-02, -1.0351e-02, -6.5548e-02,  3.5765e-02, -4.5352e-02,
         -2.6277e-02,  4.6952e-02,  1.5202e-02, -1.5501e-02,  3.8304e-04,
         -1.7497e-02, -6.0437e-02,  4.3994e-03, -6.8155e-02, -5.6639e-02,
         -2.4169e-02,  4.3117e-02,  5.2147e-03, -1.5196e-02,  9.7841e-03,
         -6.5523e-02,  4.3005e-02,  6.0902e-02,  2.5703e-03, -7.8369e-02,
          7.6009e-02,  8.4996e-03, -2.1377e-02, -4.9307e-02,  1.1909e-01,
         -6.4489e-02,  8.6670e-02,  3.0398e-02, -6.8697e-02,  7.8314e-02,
          4.8507e-02, -6.3483e-03, -1.9889e-02, -8.8189e-03,  9.0872e-02,
          1.2882e-02, -6.9134e-02,  4.

In [3]:
import torch
from tqdm.auto import tqdm
def get_features_chunked(waveform: torch.Tensor, sampling_rate: int, chunk_size_seconds: float=0.25) -> torch.Tensor:
    if len(waveform.shape) != 1:
        raise ValueError("waveform should have shape [num_samples]")
    chunk_size = int(8 * ((sampling_rate * chunk_size_seconds) // 8))
    all_features = []
    for offset in tqdm(range(0, waveform.shape[0], chunk_size)):
        chunk = waveform[offset:offset+chunk_size]
        inputs = feature_extractor(chunk, sampling_rate=sampling_rate, return_tensors="pt")
        print(inputs.keys())
        audio_features = model.get_audio_features(input_features=inputs.input_features.to(device))
        all_features.append(audio_features)
        
    return torch.stack(all_features)

In [4]:
get_features_chunked(waveform[0], sampling_rate=sampling_rate)

  0%|          | 0/720 [00:00<?, ?it/s]

dict_keys(['input_features', 'is_longer'])


NotImplementedError: The operator 'aten::upsample_bicubic2d.out' is not currently implemented for the MPS device. If you want this op to be added in priority during the prototype phase of this feature, please comment on https://github.com/pytorch/pytorch/issues/77764. As a temporary fix, you can set the environment variable `PYTORCH_ENABLE_MPS_FALLBACK=1` to use the CPU as a fallback for this op. WARNING: this will be slower than running natively on MPS.